In [1]:
#Dice loss
def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)
#IoU
# from keras import backend as K
# def iou(y_true, y_pred, smooth = 1):


In [2]:
from keras.preprocessing.image import ImageDataGenerator
def adjustData(img,mask, num_class = 23):
  img = img.astype(np.float32)
  img/= 255.0
  new_mask = np.zeros((mask.shape[0],mask.shape[1], mask.shape[2], num_class))
  for i in range(mask.shape[0]):
    this_mask = mask[i,:,:,0]
   
    for c in range(num_class):
      new_mask[i,:,:,c] = (this_mask == c).astype('int')

    return img, new_mask

def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "rgb",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 23,save_to_dir = None,target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
       
        img,mask = adjustData(img,mask,num_class)
        yield (img,mask)
def valGenerator(val_path,image_folder = 'image', mask_folder= 'mask' ,num_class = 23,image_color_mode = 'rgb',mask_color_mode = 'grayscale',
                  batch_size = 5,target_size = (256,256),as_gray = True, seed = 1):
  
  image_datagen = ImageDataGenerator()
  mask_datagen  = ImageDataGenerator()
  image_generator = image_datagen.flow_from_directory(val_path, 
                                                      classes = [image_folder],
                                                      color_mode = image_color_mode,
                                                      class_mode = None,
                                                      target_size = target_size,
                                                      batch_size = batch_size,
                                                      seed = seed)
  mask_generator = mask_datagen.flow_from_directory(val_path, 
                                classes = [mask_folder],
                                color_mode = mask_color_mode, 
                                class_mode = None, 
                                target_size = (256,256),
                                batch_size = batch_size,
                                seed = seed
                                )
  val_generator = zip(image_generator, mask_generator)
  for (img,mask) in val_generator:
    img,mask = adjustData(img, mask, num_class = num_class)
    yield (img, mask)




Using TensorFlow backend.


In [3]:
args = dict(rotation_range = 0.05, width_shift_range = 0.05,height_shift_range = 0.05,
            shear_range=0.05, zoom_range = 0.05)
#args = dict()
train_generator_ = trainGenerator(5, '/content/drive/My Drive/data/train/', 'image','mask', args,num_class = 23 ,image_color_mode='grayscale' )
val_generator_ = valGenerator('/content/drive/My Drive/data/test/','image', 'mask',23 , image_color_mode='grayscale') 

In [4]:
from keras.models import *
from keras.layers import *
from keras.applications.vgg16 import VGG16 as vgg16
def pretrained_encoder_unet( input_shape = (512,512,3)):
  encoder = vgg16(input_shape = (512,512,3),include_top = False, weights = 'imagenet')
  encoder.trainable = False  

  #EXPANSIVE PART
  #(16,16,512)
  up1 =Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(encoder.layers[18].output) 
  #(32,32,256)
  up1 = concatenate([up1, encoder.layers[17].output])# up1(32,32,256)  concatenate with (32,32,512)  = (32,32,768)
  conv1 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up1) # (32,32,256)
  drop1 = Dropout(0.2)(conv1)
  conv1 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(drop1)

  #(32,32,256)
  up2 = Conv2DTranspose(128, (2,2), strides = (2,2), padding = 'same')(conv1)
  #(64,64,128)
  up2 = concatenate([up2, encoder.layers[13].output ]) # up2(64,64,128) concatenate with (64,64,512) = (64,64,640)
  conv2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
  #(64,64,128)
  drop2 = Dropout(0.4)(conv2)
  conv2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(drop2)

  #(64,64,128)
  up3 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv2)
  #(128,128,64)
  up3 = concatenate([up3,encoder.layers[9].output]) #up3(128,128,64) concatenate with (128,128,256) = (128,128,320)
  conv3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up3)
  drop3 = Dropout(0.1)(conv3)
  conv3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(drop3)

  #(128,128,64)
  up4 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv3)
  #(256,256, 32)
  up4 = concatenate([up4,encoder.layers[5].output ]) #up4(256,256,32) concatenate with (256,256,128) = (256,256,160)
  conv4 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up4)
  drop4 = Dropout(0.1)(conv4)
  conv4 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(drop4)

  #(256,256,32)
  up5 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv4)
  #(512,512,23)
  up5 = concatenate([up5,encoder.layers[2].output]) #up5(512,512,16) concatenate with (512,512,64) = (512,512,80)
  conv5 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up5)
  drop5 = Dropout(0.1)(conv5)
  conv5 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(drop5)

  #(512,512,32)

  outputs = Conv2D(23,(1,1), activation = 'softmax')(conv5)
  model = Model(input= [encoder.layers[0].output], output = [outputs])
  for i, layer in enumerate(model.layers):
    if i< 19:
      model.layers[i].trainable = False
  model.compile(optimizer ='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  model.summary()
  return model

In [5]:

unet = pretrained_encoder_unet()

58892288/58889256 [==============================] - 5s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 512, 512, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 512, 512, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 256, 256, 64) 0           block1_conv2[0][0]               
___________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


In [6]:
from keras.callbacks import ModelCheckpoint
model_checkpoint = ModelCheckpoint(filepath = '/content/model.ckpt',
                                   save_weights_only =True,
                                   monitor = 'val_accuracy',
                                   mode = 'max',
                                   save_best_only =  True)

In [ ]:

history = model2.fit(train_generator_, epochs =10,steps_per_epoch= 100, 
                              callbacks = [model_checkpoint], validation_data = val_generator_, validation_steps = 10)



In [9]:
import matplotlib.pyplot as plt
import numpy as np
b = next(train_generator_)

fig, axis = plt.subplots(1,2,figsize = (12,9))
axis[0].imshow(b[0][0,:,:,0])
axis[1].imshow(np.argmax(b[1][0,:,:,:,], axis = 2))

StopIteration: ignored